In [38]:
import pandas as pd
import numpy as np

#on testera principalement random forest et logistic regression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline

from __future__ import division
from collections import Counter

#Importer la frame

path = '../data/merge.csv'
df = pd.read_csv(path, index_col=0)

#Traiter la frame
df.dep = df.dep.apply(lambda x: x.replace('2A', '20').replace('2B', '20'))

df.head()

,dep,circ,year,pop,dens,age_1,age_2,age_3,age_4,etud,...,presid_ED,presid_EG,presid_G,presid_autre,l_C,l_D,l_ED,l_EG,l_G,l_autre
0,1,1.0,1993.0,488244.0,85.0,29.0,30.0,24.0,18.0,0.64,...,16.09,12.18,31.41,0.0,0.00,46.47,10.28,19.15,24.11,0.0
1,1,2.0,1993.0,488244.0,85.0,29.0,30.0,24.0,18.0,0.64,...,16.09,12.18,31.41,0.0,0.00,46.53,18.79,20.73,13.95,0.0
2,1,3.0,1993.0,488244.0,85.0,29.0,30.0,24.0,18.0,0.64,...,16.09,12.18,31.41,0.0,51.59,0.00,12.41,18.83,17.18,0.0
3,1,4.0,1993.0,488244.0,85.0,29.0,30.0,24.0,18.0,0.64,...,16.09,12.18,31.41,0.0,48.86,0.00,17.46,17.72,15.95,0.0
4,2,1.0,1993.0,538219.0,73.0,29.0,29.0,22.0,19.0,0.93,...,13.41,16.56,39.55,0.0,0.00,42.65,12.33,21.31,23.70,0.0


In [2]:
#Stockons dès maintenant une binarisation des départements
#garder l'année au cas où

ddeps = pd.get_dummies(df.dep).join(df.year)
ddeps.head()

,1,10,11,12,13,14,15,16,17,18,...,92,93,94,95,971,972,973,974,976,year
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1993.0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1993.0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1993.0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1993.0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1993.0


In [33]:
groups = ['EG', 'G', 'C', 'D', 'ED', 'autre']
lgroups = ['l_%s' % g for g in groups]
pgroups = ['presid_%s' % g for g in groups]

#on stocke le "gagnant" de chaque circonscription
df['l_win'] = df[lgroups].idxmax(axis=1)

In [3]:
#On test des classifications en utilisant juste les présidentielles
X = df[pgroups]
y = df.l_win

In [4]:
#gridsearch sur plusieurs hyperparamètres. Pour régression logistique, on test différentes forces de pénalisation
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000] }

clf = GridSearchCV(LogisticRegression(), param_grid)

clf.fit(X, y)

clf.cv_results_

#manifestement c'est la plus forte pénalisation (C=0.001) qui donne le meilleur std test score... 0.44 de précision

{'mean_fit_time': array([ 0.05565508,  0.03636726,  0.04080391,  0.04184739,  0.04290032,
         0.04259205,  0.04279065]),
 'mean_score_time': array([ 0.01329557,  0.00205533,  0.00203737,  0.00200868,  0.00204102,
         0.00206168,  0.00201035]),
 'mean_test_score': array([ 0.44676471,  0.42529412,  0.42647059,  0.425     ,  0.425     ,
         0.425     ,  0.425     ]),
 'mean_train_score': array([ 0.58124111,  0.58344953,  0.58344888,  0.58344888,  0.58344888,
         0.58344888,  0.58344888]),
 'param_C': masked_array(data = [0.001 0.01 0.1 1 10 100 1000],
              mask = [False False False False False False False],
        fill_value = ?),
 'params': ({'C': 0.001},
  {'C': 0.01},
  {'C': 0.1},
  {'C': 1},
  {'C': 10},
  {'C': 100},
  {'C': 1000}),
 'rank_test_score': array([1, 3, 2, 4, 4, 4, 4], dtype=int32),
 'split0_test_score': array([ 0.27112676,  0.25616197,  0.25528169,  0.25528169,  0.25528169,
         0.25528169,  0.25528169]),
 'split0_train_score': array([ 

In [5]:
#Testons en limitant le nombre d'annees puisque l'apport des résultats de 2017 dans le jeu d'entraînement n'améliore pas
#les scores de validation

X2 = df[df.year >= 2012.0][pgroups]
y2 = df[df.year >= 2012.0].l_win

In [6]:
#On répète le grid search avec seulement 2012 et 2017 comme données.
#Cette fois en testant aussi la pénalisation l1, et avec surtout des petits C puisqu'on dirait une forte pénalisation
#est nécessaire.

param_grid = {'penalty': ['l1', 'l2'], 'C': [0.0001, 0.0005, 0.001, 0.005, 0.01] }

clf = GridSearchCV(LogisticRegression(), param_grid, cv=10)

clf.fit(X2, y2)

for i in np.argsort(clf.cv_results_['rank_test_score'])[:10]:
    print clf.cv_results_['params'][i]
    print clf.cv_results_['mean_test_score'][i]
    print '__'

{'penalty': 'l1', 'C': 0.01}
0.670250896057
__
{'penalty': 'l2', 'C': 0.001}
0.668458781362
__
{'penalty': 'l2', 'C': 0.005}
0.666666666667
__
{'penalty': 'l2', 'C': 0.01}
0.666666666667
__
{'penalty': 'l2', 'C': 0.0005}
0.661290322581
__
{'penalty': 'l1', 'C': 0.005}
0.660394265233
__
{'penalty': 'l2', 'C': 0.0001}
0.655913978495
__
{'penalty': 'l1', 'C': 0.0005}
0.631720430108
__
{'penalty': 'l1', 'C': 0.001}
0.621863799283
__
{'penalty': 'l1', 'C': 0.0001}
0.385304659498
__


En limitant le nombre d'années, on limite les différentes situations politiques, d'où les meilleurs scores. C'est sûrement de l'overfit mais on pensera au moins à supprimer les années 93 et 97 de l'entraînement plus tard. Finalement ces deux années n'ont vraiment rien à voir avec le reste des élections législatives puisqu'elles ne voient pas d'élection présidentielle se dérouler.

Partons sur des tests similaires pour Random Forest.

In [7]:
#RF a l'air d'exiger un min_samples_split un minimum élevé pour éviter l'overfit ?
#On test aussi le nombre d'arbres et le critère de séparation.

#param_grid = {'n_estimators': [10, 50, 100], 'min_samples_split': [100, 200, 300, 400, 500, 600, 700], 'criterion': ['gini', 'entropy']}
#
#clf = GridSearchCV(RandomForestClassifier(), param_grid, cv=10)
#
#clf.fit(X, y)
#
#for i in np.argsort(clf.cv_results_['rank_test_score'])[:10]:
#    print clf.cv_results_['params'][i]
#    print clf.cv_results_['mean_test_score'][i]
#    print '__'

Le gridsearch est long à effectuer pour RF. Les meilleurs scores sont : 

{'min_samples_split': 600, 'n_estimators': 100, 'criterion': 'gini'}
0.562647058824

{'min_samples_split': 600, 'n_estimators': 50, 'criterion': 'gini'}
0.557352941176

{'min_samples_split': 600, 'n_estimators': 10, 'criterion': 'gini'}
0.556764705882

{'min_samples_split': 300, 'n_estimators': 50, 'criterion': 'gini'}
0.556176470588

{'min_samples_split': 500, 'n_estimators': 50, 'criterion': 'entropy'}
0.555882352941

{'min_samples_split': 300, 'n_estimators': 10, 'criterion': 'gini'}
0.555882352941

{'min_samples_split': 600, 'n_estimators': 100, 'criterion': 'entropy'}
0.555588235294

{'min_samples_split': 500, 'n_estimators': 10, 'criterion': 'gini'}
0.555

{'min_samples_split': 500, 'n_estimators': 50, 'criterion': 'gini'}
0.554705882353

{'min_samples_split': 700, 'n_estimators': 50, 'criterion': 'gini'}
0.553235294118


In [8]:
#Retestons avec juste les données 2012 et 2017


#param_grid = {'n_estimators': [10, 25, 50, 75, 100], 'min_samples_split': [150, 250, 350, 450], 'criterion': ['gini', 'entropy']}
#
#clf = GridSearchCV(RandomForestClassifier(), param_grid, cv=10)
#
#clf.fit(X2, y2)
#
##10 combinaisons aux meilleurs scores, et leurs scores
#
#for i in np.argsort(clf.cv_results_['rank_test_score'])[:10]:
#    print clf.cv_results_['params'][i]
#    print clf.cv_results_['mean_test_score'][i]
#    print '__'

Pour 2012 et 2017 les meilleurs scores sont :
{'min_samples_split': 250, 'n_estimators': 10, 'criterion': 'entropy'}
0.6729390681

{'min_samples_split': 250, 'n_estimators': 25, 'criterion': 'entropy'}
0.664874551971

{'min_samples_split': 250, 'n_estimators': 25, 'criterion': 'gini'}
0.662186379928

{'min_samples_split': 250, 'n_estimators': 50, 'criterion': 'entropy'}
0.660394265233

{'min_samples_split': 150, 'n_estimators': 50, 'criterion': 'entropy'}
0.656810035842

{'min_samples_split': 250, 'n_estimators': 75, 'criterion': 'gini'}
0.656810035842

{'min_samples_split': 250, 'n_estimators': 100, 'criterion': 'gini'}
0.656810035842

{'min_samples_split': 250, 'n_estimators': 75, 'criterion': 'entropy'}
0.655913978495

{'min_samples_split': 150, 'n_estimators': 100, 'criterion': 'gini'}
0.655017921147

{'min_samples_split': 250, 'n_estimators': 100, 'criterion': 'entropy'}
0.655017921147


In [9]:
#Maintenant on s'intéresse à 2007/2012


X1 = df[(df.year == 2007) | (df.year == 2012)][pgroups]
y1 = df[(df.year == 2007) | (df.year == 2012)].l_win

In [10]:
#Pour lr

param_grid = {'penalty': ['l1', 'l2'], 'C': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1, 1, 10] }

clf = GridSearchCV(LogisticRegression(), param_grid, cv=10)

clf.fit(X1, y1)

for i in np.argsort(clf.cv_results_['rank_test_score'])[:10]:
    print clf.cv_results_['params'][i]
    print clf.cv_results_['mean_test_score'][i]
    print '__'

{'penalty': 'l2', 'C': 0.0005}
0.675819309123
__
{'penalty': 'l2', 'C': 0.005}
0.674933569531
__
{'penalty': 'l2', 'C': 0.01}
0.674933569531
__
{'penalty': 'l1', 'C': 0.005}
0.674047829938
__
{'penalty': 'l2', 'C': 1}
0.674047829938
__
{'penalty': 'l2', 'C': 0.001}
0.673162090345
__
{'penalty': 'l2', 'C': 0.1}
0.673162090345
__
{'penalty': 'l1', 'C': 1}
0.673162090345
__
{'penalty': 'l1', 'C': 0.01}
0.669619131975
__
{'penalty': 'l1', 'C': 0.1}
0.668733392383
__


In [11]:
#Puisque scores meilleurs sur les plus gros min_samples_split, on se concentre dessus


#param_grid = {'n_estimators': [10, 25, 50, 75, 100], 'min_samples_split': [450, 550, 650, 750], 'criterion': ['gini', 'entropy']}
#
#clf = GridSearchCV(RandomForestClassifier(), param_grid, cv=10)
#
#clf.fit(X1, y1)
#
##10 combinaisons aux meilleurs scores, et leurs scores
#
#for i in np.argsort(clf.cv_results_['rank_test_score'])[:10]:
#    print clf.cv_results_['params'][i]
#    print clf.cv_results_['mean_test_score'][i]
#    print '__'

10 meilleurs scores :

{'min_samples_split': 450, 'n_estimators': 100, 'criterion': 'entropy'}
0.651018600531

{'min_samples_split': 450, 'n_estimators': 10, 'criterion': 'entropy'}
0.647475642161

{'min_samples_split': 450, 'n_estimators': 75, 'criterion': 'entropy'}
0.646589902569

{'min_samples_split': 450, 'n_estimators': 50, 'criterion': 'entropy'}
0.645704162976

{'min_samples_split': 450, 'n_estimators': 75, 'criterion': 'gini'}
0.643932683791

In [12]:
#Travail de réintégration des variables. Une variable supplémentaire

#LR1 et RF1 correspondent aux meilleurs hyperparamètres pour s'entraîner sur 2007 et 2012.
#LR2 et RF2 pour s'entraîner sur 2012 et 2017.

#Manifestement pour RF1 il faut ajouter age_3. Pour RF2 age_4.
#Pour LR1 rien à faire.

LR1 = LogisticRegression(C=0.0005, penalty='l2')
LR2 = LogisticRegression(C=0.01, penalty='l1')

RF1 = RandomForestClassifier(min_samples_split=450, n_estimators=100, criterion='entropy')
RF2 = RandomForestClassifier(min_samples_split=250, n_estimators=75, criterion='entropy')

l1 = {}
l2 = {}

for col in df.columns[:-12]:
    l1[col] = X1.join(df[col], how='inner')
    l2[col] = X2.join(df[col], how='inner')

print "Pour 2007/2012, seul :"
print "LR : %0.5f" % np.mean(cross_val_score(LR1, X1, y1, cv=10))
print "RF : %0.5f" % np.mean(cross_val_score(RF1, X1, y1, cv=10))
for col in df.columns[:-12]:
    print "Pour 2007/2012, LR, %s réintégrée : %0.5f" % (col, np.mean(cross_val_score(LR1, l1[col], y1, cv=10)))
    print "Pour 2007/2012, RF, %s réintégrée : %0.5f" % (col, np.mean(cross_val_score(RF1, l1[col], y1, cv=10)))

print '_______'
print "Pour 2012/2017, seul :"
print "LR : %0.5f" % np.mean(cross_val_score(LR2, X2, y2, cv=10))
print "RF : %0.5f" % np.mean(cross_val_score(RF2, X2, y2, cv=10))
for col in df.columns[:-12]:
    print "Pour 2012/2017, LR, %s réintégrée : %0.5f" % (col, np.mean(cross_val_score(LR2, l2[col], y2, cv=10)))
    print "Pour 2012/2017, RF, %s réintégrée : %0.5f" % (col, np.mean(cross_val_score(RF2, l2[col], y2, cv=10)))

Pour 2007/2012, seul :
LR : 0.67642
RF : 0.64100
Pour 2007/2012, LR, dep réintégrée : 0.67288
Pour 2007/2012, RF, dep réintégrée : 0.64457
Pour 2007/2012, LR, circ réintégrée : 0.67198
Pour 2007/2012, RF, circ réintégrée : 0.65524
Pour 2007/2012, LR, year réintégrée : 0.67642
Pour 2007/2012, RF, year réintégrée : 0.66244
Pour 2007/2012, LR, pop réintégrée : 0.66145
Pour 2007/2012, RF, pop réintégrée : 0.65347
Pour 2007/2012, LR, dens réintégrée : 0.66752
Pour 2007/2012, RF, dens réintégrée : 0.64633
Pour 2007/2012, LR, age_1 réintégrée : 0.67372
Pour 2007/2012, RF, age_1 réintégrée : 0.64902
Pour 2007/2012, LR, age_2 réintégrée : 0.67374
Pour 2007/2012, RF, age_2 réintégrée : 0.63652
Pour 2007/2012, LR, age_3 réintégrée : 0.67642
Pour 2007/2012, RF, age_3 réintégrée : 0.64902
Pour 2007/2012, LR, age_4 réintégrée : 0.67022
Pour 2007/2012, RF, age_4 réintégrée : 0.65080
Pour 2007/2012, LR, etud réintégrée : 0.67110
Pour 2007/2012, RF, etud réintégrée : 0.64010
Pour 2007/2012, LR, hf réin

In [13]:
#Et maintenant on teste des combinaisons en ajoutant 2 variables à la fois. Un algo à la fois. LR

ll1 = {}
ll2 = {}

for col1 in df.columns[:-12]:
    for col2 in df.columns[:-12]:
        if (col1 != col2) and ((col2, col1) not in ll1.keys()):
            ll1[(col1, col2)] = X1.join(df[col1]).join(df[col2])
        else:
            pass
        
for col1 in df.columns[:-12]:
    for col2 in df.columns[:-12]:
        if (col1 != col2) and ((col2, col1) not in ll2.keys()):
            ll2[(col1, col2)] = X2.join(df[col1]).join(df[col2])
        else:
            pass

print "Pour 2007/2012, seul : %0.6f" % np.mean(cross_val_score(LR1, X1, y1, cv=10))
for (col1, col2) in ll1.keys():
    score = np.mean(cross_val_score(LR1, ll1[col1, col2], y1, cv=10))
    if score > 0.678:
        print "Pour 2007/2012, %s, %s : %0.6f" % (col1, col2, score)
    else:
        pass

print '_______'
print "Pour 2012/2017, seul : %0.6f" % np.mean(cross_val_score(LR2, X2, y2))
for (col1, col2) in ll2.keys():
    score = np.mean(cross_val_score(LR2, ll2[col1, col2], y2))
    if score > 0.678:
        print "Pour 2012/2017, %s, %s : %0.6f" % (col1, col2, score)
    else:
        pass

Pour 2007/2012, seul : 0.676416
Pour 2007/2012, pop, age_3 : 0.680966
_______
Pour 2012/2017, seul : 0.660228


In [14]:
#Pour RF1, ajouter age_2 chom donne 64.46%
#Pour RF2, pop et hf (!) donne 0.6665 . Il y a aussi age3 etud qui donne 0.6655. Bref on remonte très légèrement au
#dessus du score de base.


#print "Pour 2007/2012, seul : %0.6f" % np.mean(cross_val_score(RF1, X1, y1, cv=10))
#for (col1, col2) in ll1.keys():
#    score = np.mean(cross_val_score(RF1, ll1[col1, col2], y1, cv=10))
#    if score > 0.65:
#        print "Pour 2007/2012, %s, %s : %0.6f" % (col1, col2, score)
#    else:
#        pass

#print '_______'
#print "Pour 2012/2017, seul : %0.6f" % np.mean(cross_val_score(RF2, X2, y2, cv=10))
#for (col1, col2) in ll2.keys():
#    score = np.mean(cross_val_score(RF2, ll2[col1, col2], y2, cv=10))
#    if score > 0.658:
#        print "Pour 2012/2017, %s, %s : %0.6f" % (col1, col2, score)
#    else:
#        pass

np.mean(cross_val_score(RF2, X2, y2, cv=10))

0.65472915242652086

In [15]:
#Enfin, on teste la sélection de features avec selectfrommodel.

#On met en place des frames qui contiennent toutes les variables de train.
xcols = df.columns
for lg in lgroups:
    xcols.remove(lg)
xcols.remove('l_win')
Xtot = df[xcols]
X1tot = df[(df.year == 2007) | (df.year == 2012)][xcols]
X2tot = df[(df.year == 2012) | (df.year == 2017)][xcols]


PipeLR1 = Pipeline([('feature_selection', SelectFromModel(LR1)),
                   ('clf', LR1)])
PipeLR2 = Pipeline([('feature_selection', SelectFromModel(LR2)),
                   ('clf', LR2)])

PipeRF1 = Pipeline([('feature_selection', SelectFromModel(RF1)),
                   ('clf', RF1)])
PipeRF2 = Pipeline([('feature_selection', SelectFromModel(RF2)),
                   ('clf', RF2)])

In [16]:
print "2007/2012, LR : %0.5f" % np.mean(cross_val_score(PipeLR1, X1tot, y1, cv=10))
print "2007/2012, RF : %0.5f" % np.mean(cross_val_score(PipeRF1, X1tot, y1, cv=10))
print '__'
print "2012/2017, LR : %0.5f" % np.mean(cross_val_score(PipeLR2, X2tot, y2, cv=10))
print "2012/2017, RF : %0.5f" % np.mean(cross_val_score(PipeRF2, X2tot, y2, cv=10))

2007/2012, LR : 0.67734
2007/2012, RF : 0.64993
__
2012/2017, LR : 0.66018
2012/2017, RF : 0.66556


In [17]:
#Pour mémoire
print np.mean(cross_val_score(LR1, X1, y1, cv=10))
print np.mean(cross_val_score(LR2, X2, y2, cv=10))
print np.mean(cross_val_score(RF1, X1, y1, cv=10))
print np.mean(cross_val_score(RF2, X2, y2, cv=10))

#Donc les scores n'évoluent pas beaucoup. Peut-être qu'il aurait mieux valu sélectionner les features avec un autre
#modèle que celui qu'on utilise pour les prédictions.

0.676415680434
0.670005126452
0.649049056382
0.65833304853


In [18]:
#Juste histoire de voir les variables choisies par les sfm :

sfm_LR1 = SelectFromModel(LR1)
sfm_LR2 = SelectFromModel(LR2)

sfm_RF1 = SelectFromModel(RF1)
sfm_RF2 = SelectFromModel(RF2)

In [19]:
sfm_LR1.fit(X1tot, y1)
Xsfm_LR1 = sfm_LR1.transform(X1tot)

sfm_LR2.fit(X2tot, y2)
Xsfm_LR2 = sfm_LR2.transform(X2tot)

In [20]:
#Ici on a juste exclu presid_autre et intégré age_4
Xsfm_LR1

array([[ 19.  ,  19.62,  34.5 ,  15.97,   9.24,  20.67],
       [ 19.  ,  19.62,  34.5 ,  15.97,   9.24,  20.67],
       [ 19.  ,  19.62,  34.5 ,  15.97,   9.24,  20.67],
       ..., 
       [ 20.  ,  11.24,  34.24,  14.17,  12.76,  27.32],
       [ 20.  ,  11.24,  34.24,  14.17,  12.76,  27.32],
       [ 20.  ,  11.24,  34.24,  14.17,  12.76,  27.32]])

In [24]:
#Ici par contre on est sur du dep, year, dens, ED, EG, G
Xsfm_LR2

array([[  1.00000000e+00,   2.01200000e+03,   1.06000000e+02, ...,
          2.29500000e+01,   1.34600000e+01,   2.27500000e+01],
       [  1.00000000e+00,   2.01200000e+03,   1.06000000e+02, ...,
          2.29500000e+01,   1.34600000e+01,   2.27500000e+01],
       [  1.00000000e+00,   2.01200000e+03,   1.06000000e+02, ...,
          2.29500000e+01,   1.34600000e+01,   2.27500000e+01],
       ..., 
       [  9.74000000e+02,   2.01700000e+03,   3.39000000e+02, ...,
          2.63500000e+01,   2.72600000e+01,   7.67000000e+00],
       [  9.76000000e+02,   2.01700000e+03,   6.25000000e+02, ...,
          3.02700000e+01,   1.13000000e+01,   4.33000000e+00],
       [  9.76000000e+02,   2.01700000e+03,   6.25000000e+02, ...,
          3.02700000e+01,   1.13000000e+01,   4.33000000e+00]])

In [27]:
sfm_RF1.fit(X1tot, y1)
Xsfm_RF1 = sfm_RF1.transform(X1tot)

sfm_RF2.fit(X2tot, y2)
Xsfm_RF2 = sfm_RF2.transform(X2tot)

In [28]:
#year, C, D, EG, G, autre
Xsfm_RF1

array([[  2.00700000e+03,   1.96200000e+01,   3.45000000e+01,
          9.24000000e+00,   2.06700000e+01,   0.00000000e+00],
       [  2.00700000e+03,   1.96200000e+01,   3.45000000e+01,
          9.24000000e+00,   2.06700000e+01,   0.00000000e+00],
       [  2.00700000e+03,   1.96200000e+01,   3.45000000e+01,
          9.24000000e+00,   2.06700000e+01,   0.00000000e+00],
       ..., 
       [  2.01200000e+03,   1.12400000e+01,   3.42400000e+01,
          1.27600000e+01,   2.73200000e+01,   2.80000000e-01],
       [  2.01200000e+03,   1.12400000e+01,   3.42400000e+01,
          1.27600000e+01,   2.73200000e+01,   2.80000000e-01],
       [  2.01200000e+03,   1.12400000e+01,   3.42400000e+01,
          1.27600000e+01,   2.73200000e+01,   2.80000000e-01]])

In [29]:
#year, C, D, EG, G, autre
Xsfm_RF2

array([[  2.01200000e+03,   1.01600000e+01,   3.04100000e+01,
          1.34600000e+01,   2.27500000e+01,   2.70000000e-01],
       [  2.01200000e+03,   1.01600000e+01,   3.04100000e+01,
          1.34600000e+01,   2.27500000e+01,   2.70000000e-01],
       [  2.01200000e+03,   1.01600000e+01,   3.04100000e+01,
          1.34600000e+01,   2.27500000e+01,   2.70000000e-01],
       ..., 
       [  2.01700000e+03,   1.89100000e+01,   1.72600000e+01,
          2.72600000e+01,   7.67000000e+00,   2.54000000e+00],
       [  2.01700000e+03,   1.92100000e+01,   3.26200000e+01,
          1.13000000e+01,   4.33000000e+00,   2.27000000e+00],
       [  2.01700000e+03,   1.92100000e+01,   3.26200000e+01,
          1.13000000e+01,   4.33000000e+00,   2.27000000e+00]])

La sélection automatique de features n'augmente pas beaucoup les scores de validation. Mais elle nous indique qu'on pourrait supprimer certains des scores de présidentielles des variables d'entraînement, puisqu'ils sont dépendants les uns des autres.

In [36]:
#quelle prediction de 2017 pour LR1 et RF1 ?

LR1.fit(X1, y1)
RF1.fit(X1, y1)

test = df[df.year == 2017][pgroups]

p_lr1 = LR1.predict(test)
p_rf1 = RF1.predict(test)

In [39]:
Counter(p_lr1)

Counter({'l_D': 57, 'l_G': 501})

In [40]:
Counter(p_rf1)

Counter({'l_D': 239, 'l_G': 319})

In [44]:
#Pas terrible comme prédiction, surtout quand on voit ce qu'il est advenu des candidats PS cette année.
#Mieux avec quelques variables réintégrées ? J'en doute...

#Pour LR1 le meilleur score obtenu était 68% en réintégrant pop et age_3
X1b = X1.join(df[['pop', 'age_3']])
LR1.fit(X1b, y1)

test2 = test.join(df[['pop', 'age_3']])

p_lr1 = LR1.predict(test2)
Counter(p_lr1)
#Pas beaucoup mieux

Counter({'l_D': 171, 'l_G': 387})

In [45]:
#Pour RF1 le meilleur score obtenu était 66.2% en réintégrant year... Peu probable que l'année ait une valeur
#explicative et le modèle va certainement s'en servir pour faire le tri entre l'année 2007 et l'année 2012.
X1c = X1.join(df['year'])
RF1.fit(X1c, y1)

test3 = test.join(df['year'])

p_rf1 = RF1.predict(test3)
Counter(p_rf1)
#On est toujours sur une prédiction binaire.

Counter({'l_D': 249, 'l_G': 309})

Le score de précision semble être un indicateur insuffisant de la pertinence des modèles. Même en jouant avec les variables d'entrée, les années utilisées pour l'entraînement et les hyperparamètres pour améliorer le score, on reste sur une prédiction qui donne l'avantage à la gauche et à la droite. Normal, c'est historiquement les deux groupes qui ont "remporté" le plus de circonscriptions. Peut-être qu'on devrait faire quelque chose pour que l'algorithme ne donne pas un avantage si prononcé à ces deux groupes ? On pourrait par exemple cloner des lignes des autres classes...